# GPIO


## 什么是GPIO？



GPIO（General-purpose input/output）即通用型输入输出，GPIO可以控制连接在其之上的引脚实现信号的输入和输出
芯片的引脚与外部设备相连，从而实现与外部硬件设备的通讯、控制及信号采集等功能

CPU本身是不能直接控制硬件的，硬件一般是由其对应的`控制器`来控制

片上系统SOC中将各个`硬件控制器的寄存器`映射到了`CPU地址空间`中的一段范围，这样CPU就可以通过读写寄存器来间接控制硬件

![alt text](image-8.png)

## LED实验



实验步骤：  

1. 通过电路原理图分析LED的控制逻辑 - 高电平点亮、低电平熄灭（跟硬件和电路设计有关）

2. 通过电路原理图查找LED与Exynos4412的连接关系 - GPX2_7

3. 通过数据手册分析GPIO中哪些寄存器可以控制LED - GPX2CON、GPX2DAT

4. 通过程序去操控对应的寄存器完成对LED的控制



In [ ]:
汇编代码实现板上LED的点亮或熄灭

.text   @ 伪指令指示编译器将下面有代码部分标识为程序的代码段         
_start:  @ 标记表示汇编代码的开始

MAIN:    
	BL LED_CONFIG    @ 跳转到LED配置程序
LOOP:
	BL LED_ON
	BL DELAY
	BL LED_OFF
	BL DELAY
	B  LOOP

LED_CONFIG:
	LDR R2, =0x11000c40   @ 将32位数放到R2寄存器  数代表GPX2CON寄存器的地址
	LDR R1, =0x10000000   @ 将GPX2CON[7] (对应[31:28]) 设置成 1 = output
	STR R1, [R2]          @ 将R1中的值写到R2
	@ 这里将GPX2_7设置成输出模式
	MOV PC, LR            @ 执行完返回跳转前

LED_ON:
	LDR R2, =0x11000c44   @ GPX2DAT[7:0]寄存器
	LDR R1, =0x00000080   @ 将第7位写1 00000000 00000000 00000000 0100000000
	STR R1, [R2]
	MOV PC, LR

LED_OFF:
	LDR R2, =0x11000c44
	LDR R1, =0x00000000   @ 将第7位写0
	STR R1, [R2]
	MOV PC, LR

DELAY:
	LDR R1, =100000000
L:
	SUB R1, R1, #1  @ R1 = R1 - 1 
	CMP R1, #0      @ do {R1 = R1 - 1;} while(R1 != 0)
	BNE L
	MOV PC, LR

STOP:
	B STOP

.end

In [ ]:
使用Makefile来编译

TARGET = led-asm  # 目标工程名
CROSS_COMPILE = arm-none-linux-gnueabi-  # 使用ARM处理器的交叉编译工具
CC = $(CROSS_COMPILE)gcc    # 变量 CC  代表编译命令
LD = $(CROSS_COMPILE)ld     # 变量 LD  代表链接命令
OBJCOPY = $(CROSS_COMPILE)objcopy  # 变量OBJCOPY  代表用objcopy工具对生成的程序进行处理以在裸机上运行
 
all:
    $(CC) -c $(TARGET).s -o $(TARGET).o   # 将汇编代码生成机器码
    $(LD) $(TARGET).o -Ttext 0x40008000 -o $(TARGET).elf  
    # 链接生成elf可执行文件 -Ttext 代表重定向链接地址 将编译后的程序放到该地址去运行，ARM内存的起始地址
    $(OBJCOPY) -O binary -S $(TARGET).elf $(TARGET).bin   
    # 指令转化成binary格式 
 
clean:
    rm $(TARGET).o $(TARGET).elf $(TARGET).bin  


## C语言实现LED实验：


一.汇编语言访问存储器

1.读存储器
 
    LDR R1,[R2]
 
2.写存储器
 
 	STR R1,[R2]

二.C语言访问存储器

1.读存储器

	data = *ADDR

2.写存储器

	*ADDR = data

In [ ]:
//延时函数
void Delay(unsigned int Time)
{
	while(Time --);
}
//主函数
int main()
{
	/*通过设置GPX2CON寄存器来将GPX2_7引脚设置成输出功能*/
	*(unsigned int *)0x11000c40 = 0x10000000;    //GPX2CON
//通过强制类型转换来让编译器知道这是一个地址 
//
/*
1. `0x11000c40`：这是一个十六进制数，表示一个内存地址。
2. `(unsigned int *)0x11000c40`：这是一个类型转换操作，将上述十六进制数转换为一个指向 `unsigned int` 类型的指针变量
	unsigned int * 可以告诉编译器，指针变量将来指向一个4字节的地址
	通过这种方式，我们将一个常量转换成了一个指针变量
3. `*(unsigned int *)0x11000c40`：这是一个解引用操作，获取该内存地址上存储的 `unsigned int` 类型的值。
4. `*(unsigned int *)0x11000c40 = 0x10000000;`：这是一个赋值操作，将值 `0x10000000`（一个十六进制数）写入到内存地址 `0x11000c40` 所指向的位置。
我们把0x10000000写入到了0x11000c40这个地址上

！！ 不同于以往我们通过申请一个内存地址用指针变量来保存，这是系统分配的，现在我们明确代码应该运行在哪段内存
*/
	while(1)
	{
		/*点亮LED2*/
		*(unsigned int *)0x11000c44 = 0x00000080;  //GPX2DAT
		//
		/*延时*/
		Delay(1000000);
		/*熄灭LED2*/
		*(unsigned int *)0x11000c44 = 0x00000000;
		/*延时*/
		Delay(1000000);
	}
	return 0;
}

上面的方法在修改寄存器地址的时候，需要写很多重复的代码

这里可以使用宏定义来简化代码

In [ ]:
#define GPX2CON (*(unsigned int *)0x11000c40)
#define GPX2DAT (*(unsigned int *)0x11000c44)
// 宏定义，使用标签提高代码的可读性
int main()
{
	GPX2CON = 0x10000000;

	while(1)
	{
		/*点亮LED2*/
		GPX2DAT = 0x00000080;
		/*延时*/
		Delay(1000000);
		/*熄灭LED2*/
		GPX2DAT = 0x00000000;
		/*延时*/
		Delay(1000000);
	}
	return 0;
}

如果需要修改的`寄存器比较多`而且`地址连续`的话，可以使用结构体

In [ ]:
typedef struct
{
	unsigned int CON;
	unsigned int DAT;
	unsigned int PUD;
	unsigned int DRV;
}gpx2;
//定义一个结构体类型gpx2，包含了GPX2CON、GPX2DAT、GPX2PUD、GPX2DRV四个寄存器

#define GPX2 (*(gpx2 *)0x11000c40)  //告诉编译器，指针向一个gpx2结构体的地址

int main()
{
	GPX2.CON = 0x10000000;

	while(1)
	{
		/*点亮LED2*/
		GPX2.DAT = 0x00000080;
		/*延时*/
		Delay(1000000);
		/*熄灭LED2*/
		GPX2.DAT = 0x00000000;
		/*延时*/
		Delay(1000000);
	}
	return 0;
}

定义GPX2相关的，甚至是芯片上所有的寄存器的结构体的定义都可以提前做好，写到头文件里

In [ ]:
#include "exynos_4412.h"  //该头文件定义好了所有的寄存器的地址

int main()
{
	GPX2.CON = 0x10000000;

	while(1)
	{
		/*点亮LED2*/
		GPX2.DAT = 0x00000080;
		/*延时*/
		Delay(1000000);
		/*熄灭LED2*/
		GPX2.DAT = 0x00000000;
		/*延时*/
		Delay(1000000);
	}
	return 0;
}

如果直接用32位16进制数来赋值，会有可能对其他位造成影响，所以这里可以用逻辑运算来只对相关的位进行操作

In [ ]:
#include "exynos_4412.h"

int main()
{
	GPX2.CON = GPX2.CON & (~(0xF << 28)) | (0x1 << 28);

	while(1)
	{
		/*点亮LED2*/
		GPX2.DAT = GPX2.DAT | (1 << 7);
		/*延时*/
		Delay(1000000);
		/*熄灭LED2*/
		GPX2.DAT = GPX2.DAT & (~(1 << 7));
		/*延时*/
		Delay(1000000);
	}
	return 0;
}

In [ ]:

  1.unsigned int a; 将a的第3位置1，其他位保持不变  
  	******** ******** ******** ********
  	******** ******** ******** ****1***
  	00000000 00000000 00000000 00001000
 
  	a = a | (1 << 3);
	
	或运算可以不对原本为0的位进行变动
 
  2.unsigned int a; 将a的第3位置0，其他位保持不变
  	******** ******** ******** ********
  	******** ******** ******** ****0***
  	11111111 11111111 11111111 11110111
 
  	a = a & (~(1 << 3));

	这里要使用与运算才能保持其他位不变
 
  	3.unsigned int a; 将a的第[7:4]位置为0101，其他位保持不变
  	******** ******** ******** ********
  	******** ******** ******** 0101****
 
  	1).先清零  把相关的位清零
	00000000 00000000 00000000 00001111  0xF
	00000000 00000000 00000000 11110000  0xF << 4
  	11111111 11111111 11111111 00001111  ~(0xF << 4)
 
  	a = a & (~(0xF << 4));
 
  	2).再置位
  	00000000 00000000 00000000 00000101  0x5 
	00000000 00000000 00000000 01010000  0x5 << 4
 
  	a = a | (0x5 << 4);
 
  	=> a = a & (~(0xF << 4)) | (0x5 << 4);


## 总结用C语言进行ARM4421芯片的开发方法

1. 查看电路原理图，查看芯片相应的引脚
2. 查看芯片手册，查看需要的寄存器
3. 引入提前定义好的头文件
4. 对寄存器内某些位进行修改的一般方法
   1. 用 & 和 （~0x nF << m） 来先对需要修改位先清零
   2. 用 | 和 （0x n << m）来对需要修改的位进行置一  
   3. a = a & (~(0xF << 4)) | (0x5 << 4)

# UART

UART  ：Universal Asynchronous Receiver Transmitter 即通用异步收发器

是一种通用的`串行`、`异步`通信总线  

串行：使用少量的线运输多位数据
异步：

该总线有两条数据线，可以实现`全双工`的发送和接收,在嵌入式系统中常用于主机与辅助设备之间的通信  

什么是串行，什么又是并行？

并行： 

![alt text](image-9.png)

串行： 

![alt text](image-10.png)

什么是单工通信，什么又是双工通信？

单工通信：  

![alt text](image-11.png)

双工通信：

![alt text](image-12.png)

波特率：波特率用于描述UART通信时的通信速度，其单位为bps(bit per second)即每秒钟传送的bit的数量。（每秒传送二进制位的个数）

UART帧格式：

![alt text](image-13.png)

空闲时是高电平，起始位是低电平。

串口协议规定发送数据时，先发送低位，串口可发5位，不多于8位。

校验位可有可无，有时会影响速度，无会影响安全。原理：奇偶校验。

停止位是高电平，可以是1或1.5或2位。


如何分辨0与00,1与11？

利用波特率，时间上判断。


如何避免发收双方时间有累积误差？

串口通信不允许连续发送，每次通信最多发送一个字节（8位）。发送完，重新通信发送新数据，时间重新计算。


## Exynos4412下的UART控制器

1. 硬件连接：

![alt text](image-14.png)

CON7里真正有用的就两个分别为TXD/RXD（14/13），为了增强抗干扰能力，加入SP3232EEA芯片。

SP3232EEA芯片的作用：将串口发出的TTL信号转转成232信号。转换后通信能力增强。

![alt text](image-15.png)

在主芯片上TXD/RXD对应AD25与AC20引脚。

2. 通信的控制：

首先需要对引脚的功能进行设置。如上图所示，需要对GPIO中的GPA1_0和GPA1_1进行设置，将其切换到UART功能。

![alt text](image-16.png)

通过查阅手册，知道了可以通过控制GPA1CON[0]和GPA1CON[1]来对GPA1_0和GPA1_1进行控制

![alt text](image-18.png)

可以通过对4412的相关寄存器来完成对通信的控制

查阅芯片手册后，可以知道我们需要的寄存器有：  

ULCONn 用于控制串口的功能，包括对数据帧格式的控制  

UCONn  设置发送和接收的模式等

In [ ]:
#include "exynos_4412.h"

void UART_Init(void)
{
	
/*1.将GPA1_0和GPA1_1设置成UART2的接收和发送引脚 GPA1CON[7:0]*/

	/*
	查表可知GPA1_0和GPA1_1 分别由GPA1CON寄存器的[7:0]来控制
	当对[3:0]设置0x2代表设置该端口功能模式为UART_2_RXD
	**** **** **** **** **** **** **** **10
	当对[7:4]设置0x2代表设置该端口功能模式为UART_2_TXD
	**** **** **** **** **** **** **10 ****
	*/
	GPA1.CON = GPA1.CON & (~(0xFF << 0)) | (0x22 << 0);
	/*
	1111 1111 1111 1111 1111 1111 0000 0000  ~0xFF
	0000 0000 0000 0000 0000 0000 0010 0010  0x22
	1111 1111 1111 1111 1111 1111 0010 0010  (~(0xFF << 0)) | (0x22 << 0)
	*/

/*2.设置UART2的帧格式 8位数据位 1位停止位 无校验 正常模式 ULCON2[6:0]*/

	UART2.ULCON2 = UART2.ULCON2 & (~(0x7F << 0)) | (0x3 << 0);
	/*
	0000 0000 0000 0000 0000 0000 0111 1111  0x7F
	1111 1111 1111 1111 1111 1111 1000 0000  ~0x7F
	0000 0000 0000 0000 0000 0000 0000 0011  0x3
	1111 1111 1111 1111 1111 1111 1000 0011  (~(0x7F << 0)) | (0x3 << 0)
	*/

/*3.设置UART2的接收和发送模式为轮询模式 UCON2[3:0]*/
	UART2.UCON2 = UART2.UCON2 & (~(0xF << 0)) | (0x5 << 0);
	/*
	0000 0000 0000 0000 0000 0000 0000 1111  0xF
	1111 1111 1111 1111 1111 1111 1111 0000  ~0xF
	0000 0000 0000 0000 0000 0000 0000 0101  0x5
	1111 1111 1111 1111 1111 1111 1111 0101  (~(0xF << 0)) | (0x5 << 0)
	*/
	
/*4.设置UART2的波特率为115200 UBRDIV2/UFRACVAL2*/
	UART2.UBRDIV2 = 53;  //115200 = 66000000 / (16 * (UBRDIV2 + UFRACVAL2 / 16))
	UART2.UFRACVAL2 = 4;  //115200 = 66000000 / (16 * (53 + 4 / 16))
}

void UART_Send_Byte(char Dat)  // 发送一个字符
{
	/*等待发送寄存器为空，即上一个数据已经发送完成 UTRSTAT2[1]*/
	while(!(UART2.UTRSTAT2 & (1 << 1)));   //等待发送寄存器为空,否则等待 
	/*将要发送的数据写入发送寄存器 UTXH2*/
	UART2.UTXH2 = Dat;
}

char UART_Rec_Byte(void)  //接收一个字符
{
	char Dat = 0;
	/*判断接收寄存器是否接收到了数据 UTRSTAT2[0]*/
	if(UART2.UTRSTAT2 & 1)  //如果接收到数据则置1
	{
		/*从接收寄存器中读取接收到的数据 URXH2*/
		Dat = UART2.URXH2;
		return Dat;
	}
	else
	{
		return 0;
	}
}

void UART_Send_Str(char * pstr)  //发送字符串
{
	while(*pstr != '\0')
		UART_Send_Byte(*pstr++);
}

int main()
{
	char RecDat = 0;
	UART_Init();

	while(1)
	{
		/*
		RecDat = UART_Rec_Byte();
		if(RecDat == 0)
		{
			
		}
		else
		{
			RecDat = RecDat + 1;
			UART_Send_Byte(RecDat);
		}
		*/
		
		/*
		UART_Send_Str("Hello World\n");
		*/

		printf("Hello World\n");
	}
	return 0;
}

代码实现在终端输入“2”，点亮LED2，输入其他字符，关闭LED2

In [ ]:
#include "exynos_4412.h"
 
void UART_Init(void)
{
	/*1.将GPA1_0和GPA1_1设置成UART2的接收和发送引脚 GPA1CON[7:0]*/
	GPA1.CON = GPA1.CON & (~(0xFF << 0)) | (0x22 << 0);
	/*2.设置UART2的帧格式 8位数据位 1位停止位 无校验 正常模式 ULCON2[6:0]*/
	UART2.ULCON2 = UART2.ULCON2 & (~(0x7F << 0)) | (0x3 << 0);
	/*3.设置UART2的接收和发送模式为轮询模式 UCON2[3:0]*/
	UART2.UCON2 = UART2.UCON2 & (~(0xF << 0)) | (0x5 << 0);
	/*4.设置UART2的波特率为115200 UBRDIV2/UFRACVAL2*/
	UART2.UBRDIV2 = 53;
	UART2.UFRACVAL2 = 4;
}
 
void UART_Send_Byte(int Dat)
{
	/*等待发送寄存器为空，即上一个数据已经发送完成 UTRSTAT2[1]*/
	while(!(UART2.UTRSTAT2 & (1 << 1)));
	/*将要发送的数据写入发送寄存器 UTXH2*/
	UART2.UTXH2 = Dat;
}
 
int UART_Rec_Byte(void)
{
	int Dat = 0;
	/*判断接收寄存器是否接收到了数据 UTRSTAT2[0]*/
	if(UART2.UTRSTAT2 & 1)
	{
		/*从接收寄存器中读取接收到的数据 URXH2*/
		Dat = UART2.URXH2;
		return Dat;
	}
	else
	{
		return 0;
	}
}
 
void UART_Send_Str(char * pstr)
{
	while(*pstr != '\0')
		UART_Send_Byte(*pstr++);
}
 
int main()
{
	int RecDat = 0;
    int flag = 0;
	UART_Init();
    GPX2.CON = GPX2.CON & (~(0xF << 28)) | (0x1 << 28); //2_7[31:28]
 
	while(1)
	{
		RecDat = UART_Rec_Byte();
 
		if(RecDat == '2')
		{
            flag = ~flag;
            if(flag)
            {
                /*点亮LED2*/
		        GPX2.DAT = GPX2.DAT | (1 << 7);
		        /*延时*/
            }
            else
		    {
                /*熄灭LED2*/
		        GPX2.DAT = GPX2.DAT & (~(1 << 7));
            }
		}	
	}
	return 0;
}

# WDT

Watch Dog Timer即看门狗定时器，其主要作用是当发生软件故障时可产生复位信号使SOC复位，其本质是一个计数器

![alt text](image.png)

WDT工作原理：

![alt text](image-1.png)

定时器预先设置一个数，这个数会不断递减，直到减到零，然后CPU会重置。  
CPU正常使用时一般会定时“喂狗”，把预设的数再设大一点，以确保不会减到零。如果CPU遇到问题了，则无法定时喂狗，一段时间后CPU就会重置。  

查询手册，WDT的使用方法

![alt text](image-17.png)

定时器的循环时间为：  
 1/PCLK/(一级分频倍率+1)/二级分频倍率） 
 PCLK的数值为100M

 WDT寄存器

 ![alt text](image-19.png)

WTCON:控制看门狗寄存器的功能

![alt text](image-21.png)

WTDAT：产生中断信号

WTCNT：储存递减值

![alt text](image-20.png)

 倒计时最大数65535.[15:0]

 WDT编程实验：

In [ ]:
#include "exynos_4412.h"
 
void Delay(unsigned int Time)
{
	while(Time--);
}
 
int main()
{
	/*设置一级分频*/
	WDT.WTCON = WDT.WTCON | (0xFF << 8);
	/*设置二级分频*/
	/*WTCNT递减频率 = PLCK(100000000)/(0xFF + 1)/128 = 3052*/
	WDT.WTCON = WDT.WTCON | (0x3 << 3);
	/*禁止WDT产生中断信号*/
	WDT.WTCON = WDT.WTCON & (~(1 << 2));
	/*使能WDT产生复位信号*/
	WDT.WTCON = WDT.WTCON | 1;
	/*设置计数器的初始值*/
	WDT.WTCNT = (3052 * 5);
	/*使能WDT,计数器开始递减*/
	WDT.WTCON = WDT.WTCON | (1 << 5);
	
	while(1)
	{
		printf("WDT.WTCNT = %d\n",WDT.WTCNT);
		/*喂狗*/
		WDT.WTCNT = 3052;
		Delay(100000);
	}
 
	return 0;
}

//编程实现将WDT的递减频率设置为10000HZ，程序运行5s后开发板复位


In [ ]:
#include "exynos_4412.h"
 
void Delay(unsigned int Time)
{
	while(Time--);
}
 
int main()
{
	/*设置一级分频*/
	WDT.WTCON = WDT.WTCON | (0x4e << 8);
	/*设置二级分频*/
	/*WTCNT递减频率 = PLCK(100000000)/(0x4e + 1)/128 = 9889,取相近值10016也可以*/
	WDT.WTCON = WDT.WTCON | (0x3 << 3);
	/*禁止WDT产生中断信号*/
	WDT.WTCON = WDT.WTCON & (~(1 << 2));
	/*使能WDT产生复位信号*/
	WDT.WTCON = WDT.WTCON | 1;
	/*设置计数器的初始值*/
	WDT.WTCNT = (10000 * 5);
	/*使能WDT,计数器开始递减*/
	WDT.WTCON = WDT.WTCON | (1 << 5);
	
	while(1)
	{
 
		printf("WDT.WTCNT = %d\n",WDT.WTCNT);
		Delay(100000);
 
	}
 
	return 0;
}




# 中断

>CPU与硬件的交互方式
1. 轮询  
   
    CPU执行程序时不断地询问硬件是否需要其服务，若需要则给予其服务，若不需要一段时间后再次询问，周而复始。  

2. 中断

    CPU执行程序时若硬件需要其服务，对应的硬件给CPU发送中断信号，CPU接收到中断信号后将当前的程序暂停下来，转而去执行中断服务程序，执行完成后再返回到被打断的点继续执行。

3. DMA
   
   硬件产生数据后，硬件控制器可将产生的数据直接写入到存储器中，整个过程无需CPU的参与。

## 轮询的方式实现按键实验

> 目标是实现按下按键在终端打印按键已经按下的语句

> 分析硬件电路图

![alt text](image-38.png)

这里使用K2，分析电路图可知：K2不按是高电平1，K2按下去是低电平0

查找芯片上和UART_RING相连的引脚，需要将这个引脚切换成`输入端口`		
![alt text](image-39.png)				

设置GPX1CON寄存器，使引脚切换为输入模式

![alt text](image-40.png)		

输入状态时，读取GPX1DAT的值可以判断按键是否按下，如果按键按下，则相应的位会被置0
		
![alt text](image-41.png)        



In [ ]:
    #include "exynos_4412.h"
     
    int main()
    {
    	/*将GPX1_1设置成输入功能*/
    	GPX1.CON = GPX1.CON & (~(0xF << 4));  //4-7位清0
    	
    	while(1)
    	{
    		/*判断GPX1_1引脚的状态，即判断按键是否按下*/
    		if(!(GPX1.DAT & (1 << 1)))  //GPX1_2引脚按下，按下是0
    		{
    			printf("Key2 Pressed\n");
    			/*等待松手*/
    			while(!(GPX1.DAT & (1 << 1)));  //如果按下0，条件会一直成立等待
    		}
    		else
    		{
    			
    		}
    	}
    	return 0;
    }

## 中断方式实现按键实验

> 实现目标：  

GPIO控制器检测GPX1_1引脚上有无下降沿信号产生，如果检测到产生，GPIO会产生中断信号并发送给中断控制器，中断控制器接受GPIO控制器发来的中断信号，并管理这些信号，将这些信号转发给CPU，让CPU处理异常

> 外设层次

设置GPX1_1引脚的功能，将其切换成中断功能

![alt text](image-42.png)

设置管理中断的寄存器,让外部的硬件控制器产生一个中断信号发送给中断控制器

EXT_INT40 对应 GPX0 这组引脚， EXT_INT41 对应 GPX1 这组引脚

![alt text](image-43.png)

这个寄存器设置产生中断信号的方式，包括低电平，高电平，下降沿触发，上升沿触发。。。

（FLTCON寄存器用于对中断信号进行滤波处理，主要防止中断信号由于机械因素产生不干净的信号（手抖），这里不采用）

接下来设置MASK寄存器来使能中断

![alt text](image-44.png)

如果中断产生时，CPU正在处理其他的中断，这里发起的中断会被忽略，可以使用 PEND 寄存器 来注册中断，让中断挂起。

产生中断时，该位自动置1，不用手动设置

![alt text](image-45.png)

> 中断控制器

![alt text](image-46.png)

>> 中断控制器的作用

+ 多个中断同时产生时可对这些中断挂起排队，然后按照优先级依次发送给CPU处理
+ 可以为每一个中断分配一个优先级
+ 一个中断正在处理时若又产生其它中断，可将新的中断挂起，待CPU空闲时再发送
+ 可以为每一个中断选择一个CPU处理
+ 可以为每一个中断选择一个中断类型（FIQ或IRQ）
+ CPU接收到中断信号后并不能区分是哪个外设产生的，此时CPU可查询中断控制器 
+ 来获取当前的中断信号是由哪个硬件产生的，然后再进行对应的处理
+ 可以打开或禁止每一个中断

>> 中断控制器管理的160个中断信号

中断控制器将所需管理的160个中断信号进行编号，称为中断号。这个中断号仅仅是区别用，不定义中断的属性

需要在中断控制器中断表中查找GPX1_1产生的中断对应的中断号是多少？

首先查看电路原理图，得知GPX1_1对应的中断是外部中断9——XEINT9

![alt text](image-47.png)

查表得知，外部中断9对应的中断号是-57

![alt text](image-48.png)

>> 中断控制器中的相关寄存器

1. ICDDCR
   
   ![alt text](image-49.png)

   相当于整个中断控制器的总开关，使用中断需要将寄存器的第0位先写1

2. ICDISER
   
   ![alt text](image-50.png)

   这个寄存器用来设置160个每个中断的打开和关闭  
   但想每个位对应管理一个中断则需要160bit，需要32bit的寄存器5个。ICDSERn(n=0-4)这5个寄存器恰好可以管理160个中断

    ![alt text](image-51.png)

3. ICDIPTR

    设置每个中断交由哪个CPU处理

    设置方法：

    ![alt text](image-52.png)

    选择CPU需要8位数字，想要给160个中断，每个都指定用哪个CPU，需要160 x 8 = 1280bit，对应需要40个寄存器(0-39)

    ![alt text](image-53.png)

    40个寄存器和160个中断信号的对应关系：

    ![alt text](image-54.png)

4. ICCICR

    ![alt text](image-55.png)

    打开中断控制器和某个CPU的接口

代码结构分析：

interface.c 是主函数框架

common - 已经写好的头文件和源文件

start下的start.s 是ARM上电启动需要执行的汇编程序（栈初始化等）

map.lds 是链接脚本文件 保存链接的信息, 指定生成的代码的排版

In [ ]:
.text
.global _start
_start:
	/*
	 * Vector table 异常向量表，这里使用8条跳转指令来占用内存
	 */ 
	b reset  //跳转到reset
	b .
	b .
	b .
	b .
	b .
	/*
	 * 从异常向量表再跳转到IRQ的异常处理程序
	 */	
	b irq_handler
	b .

reset:
	/*
	 * Set vector address in CP15 VBAR register
	 */ 
	ldr	r0, =_start  @ 把异常向量表的地址交给r0
	mcr	p15, 0, r0, c12, c0, 0	@Set VBAR 修改异常向量表的位置
	@ ARM遇到异常跳转到0地址，可以修改到别的位置
	@ mcr协处理器指令(把ARM寄存器内的值写到协处理器中) 
	@ 把r0 寄存器内的值写入到协处理器cp15中的c12寄存器
	/*
	执行完这条指令，ARM再遇到异常就不会跳转到0，而是跳转到r0(这里自己设置的异常向量表)的地址
	*/
	/*
	 * Set the cpu to SVC32 mode, Disable FIQ/IRQ
	 * CPU刚启动不希望被打断
	 */  
	mrs r0, cpsr             @ 通过状态寄存器写入指令
	bic r0, r0, #0x1f
	orr	r0, r0, #0xd3
	msr	cpsr ,r0

	/*
	 * Defines access permissions for each coprocessor
	 */  
    mov	r0, #0xfffffff
    mcr	p15, 0, r0, c1, c0, 2  	

	/*
	 * Invalidate L1 I/D                                                                                                                   
	 */
	mov	r0, #0					@Set up for MCR
	mcr	p15, 0, r0, c8, c7, 0	@Invalidate TLBs    @页表用
	mcr	p15, 0, r0, c7, c5, 0	@Invalidate icache   
	
	/*
	 * Set the FPEXC EN bit to enable the FPU  使能运算浮点数据的协处理器
	 */ 
	mov r3, #0x40000000
	fmxr FPEXC, r3
	
	/*
	 * Disable MMU stuff and caches
	 * MMU负责物理内存和虚拟内存的转换，这里没有操作系统不用虚拟内存
	 */
	mrc	p15, 0, r0, c1, c0, 0
	bic	r0, r0, #0x00002000		@Clear bits 13 (--V-)
	bic	r0, r0, #0x00000007		@Clear bits 2:0 (-CAM)
	orr	r0, r0, #0x00001000		@Set bit 12 (---I) Icache
	orr	r0, r0, #0x00000002		@Set bit 1 (--A-) Align
	orr	r0, r0, #0x00000800		@Set bit 11 (Z---) BTB
	mcr	p15, 0, r0, c1, c0, 0

	/*
	 * Initialize stacks      初始化栈                                                                                                            
	 */
init_stack:     
	/*svc mode stack*/
	msr cpsr, #0xd3
	ldr sp, _stack_svc_end

	/*undef mode stack*/
	msr cpsr, #0xdb
	ldr sp, _stack_und_end

	/*abort mode stack*/	
	msr cpsr,#0xd7
	ldr sp,_stack_abt_end

	/*irq mode stack*/	
	msr cpsr,#0xd2
	ldr sp, _stack_irq_end
	
	/*fiq mode stack*/
	msr cpsr,#0xd1
	ldr sp, _stack_fiq_end
	
	/*user mode stack, enable FIQ/IRQ*/
	msr cpsr,#0x10
	ldr sp, _stack_usr_end

	/*Call main*/
	b main
/*
* IRQ的异常处理程序
* 要写到 b main 的后面，以确保初始化不会执行中断处理程序
*/
irq_handler:
	/*
	 * 因为产生IRQ异常后ARM自动保存到LR中的返回地址是被IRQ打断的指令
     * 的下一条再下一条指令的地址，所以我们需要人为的去修正一下
	 */
	sub lr, lr, #4
	/*
	 * 因为IRQ模式下使用的R0-R12寄存器和USER模式下使用的是同一组
	 * 所以在处理异常之前需要先将之前寄存器中的值压栈保护
	 */
	stmfd sp!, {r0-r12,lr}
	/*
	 * 跳转到do_irq处理异常
	 */
	bl do_irq
	/*
	 * 异常返回
	 * 1.将R0-R12寄存器中的值出栈，使其恢复到被异常打断之前的值
	 * 2.将SPSR寄存器中的值恢复到CPSR，使CPU的状态恢复到被异常打断之前
	 * 3.将栈中保存的LR寄存器的值出栈给PC，使程序跳转回被异常打断的点继续执行
	 */
	ldmfd sp!,{r0-r12,pc}^

_stack_svc_end:      
	.word stack_svc + 512    @ 占用1个字节
_stack_und_end:      
	.word stack_und + 512
_stack_abt_end:      
	.word stack_abt + 512
_stack_irq_end:      
    .word stack_irq + 512
_stack_fiq_end:
    .word stack_fiq + 512
_stack_usr_end:      
    .word stack_usr + 512

.data                      @ 数据段 不同于.text的代码段
stack_svc:      
	.space 512            @ 占用512个字节作栈
stack_und:
	.space 512
stack_abt:      
	.space 512
stack_irq:      
	.space 512
stack_fiq:      
	.space 512
stack_usr:      
	.space 512

In [ ]:
#include "exynos_4412.h"

void Delay(unsigned int Time)
{
	while(Time--);
}

//IRQ异常处理
void do_irq(void)
{
	unsigned int IrqNum = 0;
	/*从中断控制器中获取当前中断的中断号*/
	IrqNum = CPU0.ICCIAR & 0x3FF; 
	// 0000 0011 1111 1111  0x3FF
	// 取低10位
	
	/*根据中断号处理不同的中断*/
	switch(IrqNum)
	{
		case 0:
			//0号中断的处理程序
			break;
		case 1:
			//1号中断的处理程序
			break;
			/*
			 * ... ...
			 */
		case 57:
			printf("Key2 Pressed\n");
			/*清除GPIO控制器中GPX1_1的中断挂起标志位，不然会一直打印*/
			EXT_INT41_PEND = (1 << 1);  //这个寄存器是写1清零
			/*将当前中断的中断号写回到中断控制器中，以这种方式来告知中断控制器当前的中断已经处理完成，可以发送其它中断*/
			CPU0.ICCEOIR = CPU0.ICCEOIR & (~(0x3FF)) | (57);
			break;
			/*
			 * ... ...
			 */
		case 159:
			//159号中断的处理程序
			break;
		default:
			break;
	}

}

int main()
{
	/*外设层次 - 让外部的硬件控制器产生一个中断信号发送给中断控制器*/
	/*将GPX1_1设置成中断功能*/
	GPX1.CON = GPX1.CON | (0xF << 4);
	/*设置GPX1_1的中断触发方式为下降沿触发*/
	EXT_INT41_CON = EXT_INT41_CON & (~(0x7 << 4)) | (0x2 << 4);
	/*使能GPX1_1的中断功能*/
	EXT_INT41_MASK = EXT_INT41_MASK & (~(1 << 1));

	/*中断控制器层次 - 让中断控制器接收外设产生的中断信号并对其进行管理然后再转发给CPU处理*/
	/*全局使能中断控制器使其能接收外设产生的中断信号并转发到CPU接口*/
	ICDDCR = ICDDCR | 1;
	/*在中断控制器中使能57号中断，使中断控制器接收到57号中断后能将其转发到CPU接口*/
	ICDISER.ICDISER1 = ICDISER.ICDISER1 | (1 << 25);
	/*选择由CPU0来处理57号中断*/
	ICDIPTR.ICDIPTR14 = ICDIPTR.ICDIPTR14 & (~(0xFF << 8)) | (0X01 << 8);
	/*使能中断控制器和CPU0之间的接口，使中断控制器转发的中断信号能够到达CPU0*/
	CPU0.ICCICR = CPU0.ICCICR | 1;

	GPX2.CON = GPX2.CON & (~(0xF << 28)) | (0x1 << 28);

    while(1)
	{   
		/*点亮LED2*/
		GPX2.DAT = GPX2.DAT | (1 << 7); 
		/*延时*/
		Delay(1000000);
		/*熄灭LED2*/
		GPX2.DAT = GPX2.DAT & (~(1 << 7));
		/*延时*/
		Delay(1000000);
	}   

	return 0;
}

当某条代码产生IRQ中断后，程序会跳转到异常向量表，然后再跳转到异常处理函数
处理完异常后，程度返回到产生中断代码的下一条指令的地址，LR应该保存下一条指令的地址
PC应该指向异常向量表中IRQ指令的地址

为什么需要在中断处理程序的开始手动修正LR地址？


In [ ]:
    #include "exynos_4412.h"
     
    //IRQ异常处理
    void do_irq(void)
    {
        static int a=0;
    	unsigned int IrqNum = 0;
    	/*从中断控制器中获取当前中断的中断号*/
    	IrqNum = CPU0.ICCIAR & 0x3FF;
    	
    	/*根据中断号处理不同的中断*/
    	switch(IrqNum)
    	{
    		case 0:
    			//0号中断的处理程序
    			break;
    		case 1:
    			//1号中断的处理程序
    			break;
    			/*
    			 * 
    			 */
    		case 58:
    			printf("Key3 Pressed\n");
                if (a)
    			{
    				//点灯LED2
    				GPX2.DAT = GPX2.DAT | (1 << 7); 
    				a = !a;
    			}
    			else
    			{
    				//灭灯LED2
    				GPX2.DAT = GPX2.DAT & (~(1 << 7));
    				a = !a;
    			}
    			/*清除GPIO控制器中GPX1_2的中断挂起标志位*/
    			EXT_INT41_PEND = (1 << 2);
    			/*将当前中断的中断号写回到中断控制器中，以这种方式来告知中断控制器当前的中断已经处理完成，可以发送其它中断*/
    			CPU0.ICCEOIR = CPU0.ICCEOIR & (~(0x3FF)) | (58);
    			break;
    			/*
    			 * ... ...
    			 */
    		case 159:
    			//159号中断的处理程序
    			break;
    		default:
    			break;
    	}
     
    }
     
    int main()
    {
    	/*外设层次 - 让外部的硬件控制器产生一个中断信号发送给中断控制器*/
    	/*将GPX1_2设置成中断输入功能*/
    	GPX1.CON = GPX1.CON | (0xF << 8);
    	/*设置GPX1_2的中断触发方式为下降沿触发*/
    	EXT_INT41_CON = EXT_INT41_CON & (~(0x7 << 8)) | (0x2 << 8);
    	/*使能GPX1_2的中断功能*/
    	EXT_INT41_MASK = EXT_INT41_MASK & (~(1 << 2));
     
    	/*中断控制器层次 - 让中断控制器接收外设产生的中断信号并对其进行管理然后再转发给CPU处理*/
    	/*全局使能中断控制器使其能接收外设产生的中断信号并转发到CPU接口*/
    	ICDDCR = ICDDCR | 1;
    	/*在中断控制器中使能58号中断，使中断控制器接收到58号中断后能将其转发到CPU接口*/
    	ICDISER.ICDISER1 = ICDISER.ICDISER1 | (1 << 26);
    	/*选择由CPU0来处理58号中断*/
    	ICDIPTR.ICDIPTR14 = ICDIPTR.ICDIPTR14 & (~(0xFF << 16)) | (0X01 << 16);
    	/*使能中断控制器和CPU0之间的接口，使中断控制器转发的中断信号能够到达CPU0*/
    	CPU0.ICCICR = CPU0.ICCICR | 1;
        /*初始化LED引脚*/
    	GPX2.CON = GPX2.CON & (~(0xF << 28)) | (0x1 << 28);
    	
    	while(1)
    	;
    	return 0;
    }

# ADC

ADC(Analog to Digital Converter)即模数转换器，指一个能将模拟信号转化为数字信号的电子元件。

![alt text](image-56.png)

> ADC主要参数：分辨率    

ADC的分辨率一般以输出二进制数的位数来表示，当最大输入电压一定时，位数越高，分辨率越高； n位的ADC能区分输入电压的最小值为满量程输入的1/2^n；

比如一个12位的ADC，最大输入电压为1.8v，那么该ADC能区分的最小电压为1.8v/2^12≈0.00044v，

因此，当转换的结果为m时，则实际的电压值为m*(1.8v/2^12)；

自我理解：相当于每变化0.00044v，二进制变加或减1。

![alt text](image-57.png)

## Exynos4412下的ADC控制器实验

在开发板上的蓝色旋钮-电位器-滑动变阻器

查看电路原理图，寻找引脚连接关系

![alt text](image-58.png)

![alt text](image-59.png)


这4个引脚是专用引脚，供VDD_18_ADC，无其他功能

>> A/D 转换时间

![alt text](image-72.png)

>> 控制ADC的相关寄存器

![alt text](image-73.png)
1. ADCCON 
   
    ![alt text](image-74.png)

    RES : ADC 精度

    ECFLG ：转换完成标志位 
    
    PRSCEN：分频器开关

    PRSCVL：极限转换速度5MHz，分频时应小于此值。范围19-255

    STANDBY：待机模式选择。置0正常，置1低功耗模式

    0-1位设置AD什么时候开始转换，两种转换方式。

    READ_START：读完上次结果开始下一次AD转换，

    ENABLE_START：置1时，开始一次AD转换，转换后该位自动置0.初始化时不设置，需要转化时设置。

2. ADCDAT
   
   ![alt text](image-75.png)

3. ADCMUX
   
   ![alt text](image-76.png)

In [ ]:
#include "exynos_4412.h"
 
int main()
{
	unsigned int AdcValue;
 
	/*设置ADC精度为12bit，第16位置1*/
	ADCCON = ADCCON | (1 << 16);
	/*使能ADC分频器，第14位置1*/
	ADCCON = ADCCON | (1 << 14);
	/*设置ADC分频值 ADC时钟频率=PLCK/(19+1)=5MHZ ADC转换频率=5MHZ/5=1MHZ*/
     /*刚上电时，默认是0xFF,先清零第6-13位，再放入19*/
	ADCCON = ADCCON & (~(0xFF << 6)) | (19 << 6);
	/*关闭待机模式，使能正常模式，第2位置0*/
	ADCCON = ADCCON & (~(1 << 2));
	/*关闭通过读使能AD转换*/
    /*第0位转换时设置*/
	ADCCON = ADCCON & (~(1 << 1));
	/*选择转换通道，3通道*/
	ADCMUX = 3;
 
	while(1)
	{
		/*开始转换，设置第0位为1*/
		ADCCON = ADCCON | 1;
		/*等待转换完成，看15位是1还是0*/
		while(!(ADCCON & (1 << 15)));
		/*读取转换结果，低12位*/
		AdcValue = ADCDAT & 0xFFF;
		/*将结果转换成实际的电压值mv*/
		AdcValue = AdcValue * 0.44;
		/*打印转换结果*/
		printf("AdcValue = %dmv\n",AdcValue);
 
	}
	return 0;
}

1.编程实现通过LED状态显示当前电压范围

注：

电压在1501mv~1800mv时，LED2、LED3、LED4、LED5点亮
电压在1001mv~1500mv时，LED2、LED3、LED4点亮
电压在501mv~1000mv时，LED2、LED3点亮
电压在0mv~500mv时，LED2闪烁

In [ ]:
#include "exynos_4412.h"
 
#define LED2_ON    GPX2.DAT |= (1 << 7)
#define LED3_ON    GPX1.DAT |= 1
#define LED4_ON	   GPF3.DAT |= (1 << 4) 
#define LED5_ON	   GPF3.DAT |= (1 << 5) 
#define LED2_OFF   GPX2.DAT &= (~(1 << 7))
#define LED3_OFF   GPX1.DAT &= (~1)
#define LED4_OFF   GPF3.DAT &= (~(1 << 4))
#define LED5_OFF   GPF3.DAT &= (~(1 << 5))
 
void Delay(int time){
    while (time--);
}
int main()
{
	unsigned int AdcValue;
 
	/*设置ADC精度为12bit，第16位置1*/
	ADCCON = ADCCON | (1 << 16);
	/*使能ADC分频器，第14位置1*/
	ADCCON = ADCCON | (1 << 14);
	/*设置ADC分频值 ADC时钟频率=PLCK/(19+1)=5MHZ ADC转换频率=5MHZ/5=1MHZ*/
    /*刚上电时，默认是0xFF,先清零第6-13位，再放入19*/
	ADCCON = ADCCON & (~(0xFF << 6)) | (19 << 6);
	/*关闭待机模式，使能正常模式，第2位置0*/
	ADCCON = ADCCON & (~(1 << 2));
	/*关闭通过读使能AD转换*/
    /*第0位转换时设置*/
	ADCCON = ADCCON & (~(1 << 1));
	/*选择转换通道，3通道*/
	ADCMUX = 3;
 
    GPX2.CON = GPX2.CON & (~(0xF << 28)) | (0x1 << 28);//7[31:28]
    GPX1.CON = GPX1.CON & (~(0xF)) | (0x1);//1[3:0]
    GPF3.CON = GPF3.CON & (~(0xFF << 16)) | (0x11 << 16);//4[19:16]5[23:20]
 
	while(1)
	{
		/*开始转换，设置第0位为1*/
		ADCCON = ADCCON | 1;
		/*等待转换完成，看15位是1还是0*/
		while(!(ADCCON & (1 << 15)));
		/*读取转换结果，低12位*/
		AdcValue = ADCDAT & 0xFFF;
		/*将结果转换成实际的电压值mv，0.44=1800/4096*/
		AdcValue = AdcValue * 0.44;
		/*打印转换结果*/
		printf("AdcValue = %dmv\n",AdcValue);
 
        if((AdcValue>0)&(AdcValue<=500))
        {
            /*LED2闪烁*/
            LED3_OFF;
			LED4_OFF;
			LED5_OFF;
            LED2_ON;
			Delay(1000000);
			LED2_OFF;
			Delay(1000000);
        }
        else if((AdcValue>500)&(AdcValue<=1000))
        {
            /*LED2 ，LED3点亮*/
            LED3_OFF;
			LED4_OFF;
			LED5_OFF;
			LED2_ON;
			LED3_ON;
        }
        else if((AdcValue>1000)&(AdcValue<=1500))
        {           
            /*LED2、LED3、LED4点亮*/
            LED5_OFF;
			LED2_ON;
			LED3_ON;
			LED4_ON;
        }
        else if((AdcValue>1500)&(AdcValue<=1800))
        {
            /*LED2、LED3、LED4、LED5点亮*/
            LED2_ON;
			LED3_ON;
			LED4_ON;
			LED5_ON;
        }
     
	}
	return 0;
}

# RTC

RTC(Real Time Clock)即实时时钟，它是一个可以为系统提供精确的时间基准的元器件，RTC一般采用精度较高的晶振作为时钟源，有些RTC为了在主电源掉电时还可以工作，需要外加电池供电。

![alt text](image-60.png)

> Exynos4412下的RTC控制器

![alt text](image-61.png)

>> BCD 码

 BCD码，四位二进制表示一位十进制数

例如：

十进制数：12，二进制位：1100， BCD码：0001 0010

![alt text](image-63.png)

时钟分频器产生1HZ的时钟频率，以作为SEC

>> RTC寄存器

![alt text](image-64.png)

BCDSEC-BCDYEAR 这几个寄存器重点掌握，存储实际时间

ALMSEC-ALMYEAR 这几个寄存器用来设置预期时间

![alt text](image-65.png)

年：百十个位

![alt text](image-66.png)

月：十个位

![alt text](image-67.png)

日：十个位

![alt text](image-68.png)

星期
星期与日地址手册写反了，星期+0x007C,日+0x0080,地址手册写反了

![alt text](image-69.png)

时分秒

![alt text](image-70.png)

用于设置中断挂起标志位

![alt text](image-71.png)

RTCCON中的CTLEN用于打开RTC控制

In [ ]:
#include "exynos_4412.h"
 
int main()
{
	unsigned int OldSec = 0, NewSec = 0;
 
	/*使能RTC控制，RTCCON第0位置1*/
	RTCCON = RTCCON | 1;
	/*校准时间信息，不直接用二进制，C语言只支持10.8.16进制*/
	RTC.BCDYEAR = 0x023;
	RTC.BCDMON  = 0x12;
	RTC.BCDDAY  = 0x7;
	RTC.BCDWEEK = 0x31;
	RTC.BCDHOUR = 0x23;
	RTC.BCDMIN  = 0x59;
	RTC.BCDSEC  = 0x50;
	/*禁止RTC控制，关闭时间修改锁*/
	RTCCON = RTCCON &  (~(1));
 
	while(1)
	{
		NewSec = RTC.BCDSEC;
		if(OldSec != NewSec)
		{
			printf("20%x-%x-%x %x %x:%x:%x\n",RTC.BCDYEAR, RTC.BCDMON, RTC.BCDWEEK, RTC.BCDDAY, RTC.BCDHOUR, RTC.BCDMIN, RTC.BCDSEC);	
			OldSec = NewSec;
		}
	}
	return 0;
}

作业

编程实现通过LED状态显示当前电压范围，并打印产生低压警报时的时间
注：
电压在1501mv~1800mv时，LED2、LED3、LED4、LED5点亮
电压在1001mv~1500mv时，LED2、LED3、LED4点亮
电压在501mv~1000mv时，LED2、LED3点亮
电压在0mv~500mv时，LED2闪烁，且每隔一秒钟向终端打印一次当前的电压值及当前的时间

In [ ]:
#include "exynos_4412.h"
 
#define LED2_ON    GPX2.DAT |= (1 << 7)
#define LED3_ON    GPX1.DAT |= 1
#define LED4_ON	   GPF3.DAT |= (1 << 4) 
#define LED5_ON	   GPF3.DAT |= (1 << 5) 
#define LED2_OFF   GPX2.DAT &= (~(1 << 7))
#define LED3_OFF   GPX1.DAT &= (~1)
#define LED4_OFF   GPF3.DAT &= (~(1 << 4))
#define LED5_OFF   GPF3.DAT &= (~(1 << 5))
 
void Delay(int time){
    while (time--);
}
int main()
{
	unsigned int AdcValue;
 
	/*设置ADC精度为12bit，第16位置1*/
	ADCCON = ADCCON | (1 << 16);
	/*使能ADC分频器，第14位置1*/
	ADCCON = ADCCON | (1 << 14);
	/*设置ADC分频值 ADC时钟频率=PLCK/(19+1)=5MHZ ADC转换频率=5MHZ/5=1MHZ*/
    /*刚上电时，默认是0xFF,先清零第6-13位，再放入19*/
	ADCCON = ADCCON & (~(0xFF << 6)) | (19 << 6);
	/*关闭待机模式，使能正常模式，第2位置0*/
	ADCCON = ADCCON & (~(1 << 2));
	/*关闭通过读使能AD转换*/
    /*第0位转换时设置*/
	ADCCON = ADCCON & (~(1 << 1));
	/*选择转换通道，3通道*/
	ADCMUX = 3;
 
    GPX2.CON = GPX2.CON & (~(0xF << 28)) | (0x1 << 28);//7[31:28]
    GPX1.CON = GPX1.CON & (~(0xF)) | (0x1);//1[3:0]
    GPF3.CON = GPF3.CON & (~(0xFF << 16)) | (0x11 << 16);//4[19:16]5[23:20]
 
    unsigned int OldSec = 0, NewSec = 0;
 
	/*使能RTC控制，RTCCON第0位置1*/
	RTCCON = RTCCON | 1;
	/*校准时间信息，不直接用二进制，C语言只支持10.8.16进制*/
	RTC.BCDYEAR = 0x023;
	RTC.BCDMON  = 0x12;
	RTC.BCDDAY  = 0x7;
	RTC.BCDWEEK = 0x31;
	RTC.BCDHOUR = 0x23;
	RTC.BCDMIN  = 0x59;
	RTC.BCDSEC  = 0x50;
	/*禁止RTC控制，关闭时间修改锁*/
	RTCCON = RTCCON &  (~(1));
 
	while(1)
	{
		/*开始转换，设置第0位为1*/
		ADCCON = ADCCON | 1;
		/*等待转换完成，看15位是1还是0*/
		while(!(ADCCON & (1 << 15)));
		/*读取转换结果，低12位*/
		AdcValue = ADCDAT & 0xFFF;
		/*将结果转换成实际的电压值mv，0.44=1800/4096*/
		AdcValue = AdcValue * 0.44;
		/*打印转换结果*/
		printf("AdcValue = %dmv\n",AdcValue);
 
        NewSec = RTC.BCDSEC;
 
        if((AdcValue>0)&(AdcValue<=500))
        {
            if(OldSec != NewSec)
		    {
			printf("20%x-%x-%x %x %x:%x:%x\n",RTC.BCDYEAR, RTC.BCDMON, RTC.BCDWEEK, RTC.BCDDAY, RTC.BCDHOUR, RTC.BCDMIN, RTC.BCDSEC);	
			OldSec = NewSec;
		    }
 
            /*LED2闪烁*/
            LED3_OFF;
			LED4_OFF;
			LED5_OFF;
            LED2_ON;
			Delay(1000000);
			LED2_OFF;
			Delay(1000000);
       
        }
        else if((AdcValue>500)&(AdcValue<=1000))
        {
            /*LED2 ，LED3点亮*/
            LED3_OFF;
			LED4_OFF;
			LED5_OFF;
			LED2_ON;
			LED3_ON;
        }
        else if((AdcValue>1000)&(AdcValue<=1500))
        {           
            /*LED2、LED3、LED4点亮*/
            LED5_OFF;
			LED2_ON;
			LED3_ON;
			LED4_ON;
        }
        else((AdcValue>1500)&(AdcValue<=1800))
        {
            /*LED2、LED3、LED4、LED5点亮*/
            LED2_ON;
			LED3_ON;
			LED4_ON;
			LED5_ON;
        }
     
	}
	return 0;
}